This notebook calculates various technical indicators only using pandas and without using any technical analysis/indicator library.

In this notebook we calculate 
 - Average Directional Index (ADX)
 - Stochastic Oscillator
 - Chandelier

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

import datetime as dt
from pandas_datareader import data as pdr
yf.pdr_override()

pd.set_option('display.max_rows',None)

In [3]:
ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

stock_data=pdr.get_data_yahoo(ticker,start,end)

Enter Stock Ticker :  AAPL


AAPL
[*********************100%***********************]  1 of 1 completed


## Average Directional Index (ADX)

### Detailed Calculation

In [4]:
true_range_list = []
plus_dm_list = []
minus_dm_list = []

for i in stock_data.index:
    a,b,c = (abs(stock_data['High'][i] - stock_data['Low'][i])),(abs(stock_data['High'][i]-stock_data['Close'].shift(1)[i])),(abs(stock_data['Low'][i]-stock_data['Close'].shift(1)[i]))
    true_range = max(a,b,c)
    true_range_list.append(true_range)
    
    if ((stock_data['High'][i] - stock_data['High'].shift(1)[i])>(stock_data['Low'].shift(1)[i] - stock_data['Low'][i])):
        plus_dm = stock_data['High'][i] - stock_data['High'].shift(1)[i]
    else:
        plus_dm = 0
    plus_dm_list.append(plus_dm)     
        
    if ((stock_data['Low'].shift(1)[i] - stock_data['Low'][i])>(stock_data['High'][i] - stock_data['High'].shift(1)[i])):
        minus_dm = stock_data['Low'].shift(1)[i] - stock_data['Low'][i]        
    else:
        minus_dm = 0
    minus_dm_list.append(minus_dm)    

In [5]:
stock_data['true_range'] = true_range_list
stock_data['Plus_DM'] = plus_dm_list
stock_data['Minus_DM'] = minus_dm_list

In [6]:
stock_data['rolling_sum_Plus_DM'] = stock_data['Plus_DM'].rolling(14).sum()
stock_data['Smoothed_Plus_DM'] = (stock_data['rolling_sum_Plus_DM'] - (stock_data['rolling_sum_Plus_DM']/14)) + stock_data['rolling_sum_Plus_DM']

In [7]:
stock_data['rolling_sum_Plus_DM'] = stock_data['Plus_DM'].rolling(14).sum()
stock_data['Smoothed_Plus_DM'] = (stock_data['rolling_sum_Plus_DM'] - (stock_data['rolling_sum_Plus_DM']/14)) + stock_data['rolling_sum_Plus_DM']

In [8]:
stock_data['rolling_sum_Minus_DM'] = stock_data['Minus_DM'].rolling(14).sum()
stock_data['Smoothed_Minus_DM'] = (stock_data['rolling_sum_Minus_DM'] - (stock_data['rolling_sum_Minus_DM']/14)) + stock_data['rolling_sum_Minus_DM']

In [9]:
stock_data['rolling_sum_True_Range'] = stock_data['true_range'].rolling(14).sum()
stock_data['Smoothed_True_Range'] = (stock_data['rolling_sum_True_Range'] - (stock_data['rolling_sum_True_Range']/14)) + stock_data['rolling_sum_True_Range']

In [10]:
stock_data['Plus_DI'] = (stock_data['Smoothed_Plus_DM']/stock_data['Smoothed_True_Range'])*100
stock_data['Minus_DI'] = (stock_data['Smoothed_Minus_DM']/stock_data['Smoothed_True_Range'])*100

In [11]:
stock_data['DX'] = ((stock_data['Plus_DI'] - stock_data['Minus_DI'])/(stock_data['Plus_DI'] + stock_data['Minus_DI']))*100

In [12]:
stock_data['ADX'] = stock_data['DX'].rolling(14).mean()

In [13]:
stock_data['ADX'].describe()

count    2690.000000
mean        8.077005
std        36.988004
min       -76.705896
25%       -22.504306
50%        10.147215
75%        38.078299
max        99.857780
Name: ADX, dtype: float64

## Stochastic Oscillator

In [14]:
#ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

data_stoch=pdr.get_data_yahoo(ticker,start,end)

AAPL
[*********************100%***********************]  1 of 1 completed


In [15]:
data_stoch['L_5'] = data_stoch['Low'].rolling(5).min()
data_stoch['H_5'] = data_stoch['High'].rolling(5).max()

In [16]:
data_stoch['Fast_%K'] = ((data_stoch['Close'] - data_stoch['L_5'])/(data_stoch['H_5'] - data_stoch['L_5']))*100

In [17]:
# Fast_%D or Slow_%K
data_stoch['Slow_%K'] = data_stoch['Fast_%K'].rolling(3).mean()

In [18]:
data_stoch.head(10)

,Open,High,Low,Close,Adj Close,Volume,L_5,H_5,Fast_%K,Slow_%K
Date,,,,,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.540046,302220800,NaN,NaN,NaN,NaN
2012-01-04,14.642857,14.810000,14.617143,14.765714,12.607435,260022000,NaN,NaN,NaN,NaN
2012-01-05,14.819643,14.948214,14.738214,14.929643,12.747406,271269600,NaN,NaN,NaN,NaN
2012-01-06,14.991786,15.098214,14.972143,15.085714,12.880662,318292800,NaN,NaN,NaN,NaN
2012-01-09,15.196429,15.276786,15.048214,15.061786,12.860231,394024400,14.607143,15.276786,67.893291,NaN
2012-01-10,15.211071,15.214286,15.053571,15.115714,12.906280,258196400,14.617143,15.276786,75.581984,NaN
2012-01-11,15.095714,15.101786,14.975357,15.091071,12.885240,215084800,14.738214,15.276786,65.517217,69.664164
2012-01-12,15.081429,15.103571,14.955357,15.049643,12.849862,212587200,14.955357,15.276786,29.333349,56.810850
2012-01-13,14.989286,15.016071,14.952143,14.993214,12.801682,226021600,14.952143,15.276786,12.651103,35.833890


## Chandelier

In [19]:
#ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2021
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

data_chand=pdr.get_data_yahoo(ticker,start,end,interval='1h')

AAPL
[*********************100%***********************]  1 of 1 completed


In [20]:
true_range_list = []
for i in data_chand.index:
    a,b,c = (data_chand['High'][i] - data_chand['Low'][i]),(abs(data_chand['High'][i]-data_chand['Close'].shift(1)[i])),(abs(data_chand['Low'][i]-data_chand['Close'].shift(1)[i]))
    true_range = max(a,b,c)
    true_range_list.append(true_range)

In [21]:
data_chand['true_range'] = true_range_list

# Average/Smoothed True Range
data_chand['Average_true_range'] = data_chand['true_range'].rolling(14).mean()

In [22]:
# Highest Rolling High in last 22 days
data_chand['rolling_high_22'] = data_chand['High'].rolling(22).max()

# Highest Rolling Low in last 22 days
data_chand['rolling_low_22'] = data_chand['Low'].rolling(22).max()

In [23]:
data_chand['Chandelier_Short'] = data_chand['rolling_high_22'] - (data_chand['Average_true_range'].shift(1))*3
data_chand['Chandelier_Long'] = data_chand['rolling_low_22'] - (data_chand['Average_true_range'].shift(1))*3